### Loading data from file

파일에서 불러오는 법 <BR>

In [17]:
import numpy as np
import tensorflow as tf

In [33]:
xy = np.loadtxt('D:/jupyter/ML/data.csv',delimiter=',', dtype=np.float32)
x_data = xy[:,0:-1]
y_data = xy[:,[-1]]
# 전체의 -1 을 하면 맨 뒤만 한개 뺌
# 2차원은 [low,index]
# x_data = xy[:,0:-1]  전체 행을 가져오고 처음부터 마지막꺼 하나뺌
# y_data = xy[:,[-1]]  전체 행을 가져오고 마지막 한개만 가져온다.

In [34]:
print(x_data.shape, x_data, len(x_data))
print(y_data.shape, y_data)

(6, 3) [[ 73.  80.  75.]
 [ 93.  88.  93.]
 [ 89.  91.  90.]
 [ 96.  98. 100.]
 [ 73.  66.  70.]
 [ 53.  46.  55.]] 6
(6, 1) [[152.]
 [185.]
 [180.]
 [196.]
 [142.]
 [101.]]


In [35]:
tf.set_random_seed(777)
# seed 번호를 주어서 랜덤한 값을 뽑는 순서를 고정으로 함
# 똑같은 것을 뽑아냄
# 랜덤값을 똑같이 뽑게하고 다른 것을 변경하여 차이점을 알아보기 위해

X = tf.placeholder(tf.float32, shape=[None,3]) # x1, x2, x3 이므로
Y = tf.placeholder(tf.float32, shape=[None,1])

# W도 크기에 맞게 3 , y 값은 1개니까 1
W = tf.Variable(tf.random_normal([3,1],name='weight'))
# b는 출력값 y가 한개라서 하나
b = tf.Variable(tf.random_normal([1],name='bias'))

hypothesis = tf.matmul(X, W) + b

cost = tf.reduce_mean(tf.square(hypothesis-Y))

optimizer = tf.train.GradientDescentOptimizer(learning_rate = 1e-5)
train = optimizer.minimize(cost)

In [36]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [37]:
for step in range(2001):
    cost_val, hy_val, _ = sess.run(
        [cost,hypothesis,train],
        feed_dict={X:x_data, Y:y_data})
    if step % 10 == 0:
        print(step,"Cost : ",cost_val,
             "\nPreadiction\n",hy_val)

0 Cost :  214853.62 
Preadiction
 [[-287.52258]
 [-337.58987]
 [-336.98624]
 [-364.66562]
 [-257.30896]
 [-184.9599 ]]
10 Cost :  24.801577 
Preadiction
 [[144.60823]
 [181.9237 ]
 [174.83333]
 [192.71481]
 [138.96223]
 [107.16382]]
20 Cost :  14.880975 
Preadiction
 [[147.54991 ]
 [185.43506 ]
 [178.30597 ]
 [196.49272 ]
 [141.63748 ]
 [109.127075]]
30 Cost :  14.813848 
Preadiction
 [[147.58487]
 [185.45181]
 [178.33582]
 [196.52138]
 [141.64714]
 [109.12525]]
40 Cost :  14.747624 
Preadiction
 [[147.60008 ]
 [185.44492 ]
 [178.34233 ]
 [196.52461 ]
 [141.63878 ]
 [109.110176]]
50 Cost :  14.6818695 
Preadiction
 [[147.6151 ]
 [185.43787]
 [178.34863]
 [196.52765]
 [141.63033]
 [109.09506]]
60 Cost :  14.616482 
Preadiction
 [[147.63008]
 [185.43085]
 [178.35492]
 [196.53069]
 [141.62189]
 [109.07999]]
70 Cost :  14.55157 
Preadiction
 [[147.645  ]
 [185.42387]
 [178.3612 ]
 [196.5337 ]
 [141.61351]
 [109.06498]]
80 Cost :  14.486977 
Preadiction
 [[147.65988]
 [185.4169 ]
 [178.3674

In [38]:
# 교육을 시킨 hypothesis 로 값을 바꿔서 실행
print("Your score will be ", sess.run(hypothesis,
                feed_dict={X:[[100,70,101]]}))
# X 값을 넣어서 Y의 값이 얼마나 될지

Your score will be  [[194.08707]]


In [39]:
print("Other scores will be", sess.run(hypothesis,
                    feed_dict={X:[[60,70,110],[90,100,80]]}))
# 동시에물어보는것도 가능

Other scores will be [[184.11745]
 [169.37897]]


파일이 큰 (메모리가 큰) 것들은 numpy로 load 를 하기에는 메모리부족으로 안될 수도 있다. <br>
그래서 tersorflow 에서 Queue Runners 라는 것이 있음 <br>
여러개의 파일을 load를 해서 queue 에 쌓는다.<br>
그 다음 reader 로 연결을 하여 그거에 맞게 decoder(, 분리 등등) 를 한다.<br>
읽어온 값을 다시 queue에 쌓는다. <br>
실습할 때는 다 필요하지 않으니 어느정도의 범위만큼만 읽어와서 학습을 시킴 ( 즉, 조금씩 꺼내씀 )<br>
코딩으로 보면
1. 파일들의 리스트를 만듬<br>
filename_queue = tf.train.string_input_producer( <br>
 ['data 1.csv','data2.csv , ...] , shuffle=False, name='filename_queue')<br><br>
2. 파일을 읽어올 reader를 정리<br>
reader = tf.TextLineReader()<br>
key, value = reader.read(filename_queue)<br><br>
3. value 값을 어떻게 읽어들일 것인가?<br>
record_defaults = [[0.],[0.],[0.],[0.]]<br>
xy = tf.decode_csv(value,record_defaults=record_defaults)

4. tf.train.batch 라는 함수로 읽어들이게 된다. <br>
일종의 펌프같은 역할 <br>
train_x_batch, train_y_batch = \ <br>
    tf.train.batch([xy[0:-1], xy[-1:]], batch_size=10)<Br> 
                       x data   y data    10개씩 가지고 와라
   

for step in range(2001) : <Br>
    x_batch, y_batch = sess.run([train_x_batch, train_y_batch])<br>
   이 값을 feed_dict로 해서 넘겨주면 됨
<br> 
    ... <br>
  
coord.request_stop()<br>
coord.join(threads)

---

In [64]:
filename_queue = tf.train.string_input_producer(
 ['data.csv'] , shuffle=False, name='filename_queue')

reader = tf.TextLineReader()
key, value = reader.read(filename_queue)

record_defaults = [[0.],[0.],[0.],[0.]] #각각 데이터 타입은 이렇게 된다고 정리
xy = tf.decode_csv(value,record_defaults=record_defaults)
# 펌프로 하듯이 계속 빨아드림
train_x_batch, train_y_batch = \
    tf.train.batch([xy[0:-1], xy[-1:]], batch_size=10)
# x가 무엇인지 y가 무엇인지 정하고 펌프를 할 때 마다 몇번씩 가지고 올지를 정함

In [65]:
X = tf.placeholder(tf.float32, shape=[None,3])# shape 중요
Y = tf.placeholder(tf.float32, shape=[None,1])


W = tf.Variable(tf.random_normal([3,1],name='weight'))
b = tf.Variable(tf.random_normal([1],name='bias'))

hypothesis = tf.matmul(X, W) + b

cost = tf.reduce_mean(tf.square(hypothesis-Y))

optimizer = tf.train.GradientDescentOptimizer(learning_rate = 1e-5)
train = optimizer.minimize(cost)

In [66]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)

In [67]:
for step in range(2001):
    x_batch, y_batch = sess.run([train_x_batch,train_y_batch])
    cost_val,hy_val,_ = sess.run(
        [cost, hypothesis, train],
        feed_dict={X:x_batch, Y:y_batch})
    if step % 10 == 0:
        print(step,"Cost : ",cost_val,
             "\nPreadiction\n",hy_val)
        
coord.request_stop()
coord.join(threads)
# 결과는 똑같음

0 Cost :  4222.1865 
Preadiction
 [[ 95.67829 ]
 [111.54321 ]
 [111.55784 ]
 [122.32974 ]
 [ 83.65219 ]
 [ 62.815475]
 [ 95.67829 ]
 [111.54321 ]
 [111.55784 ]
 [122.32974 ]]
10 Cost :  8.4256 
Preadiction
 [[137.0952 ]
 [102.2141 ]
 [153.94579]
 [181.60391]
 [180.57542]
 [197.4912 ]
 [137.0952 ]
 [102.2141 ]
 [153.94579]
 [181.60391]]
20 Cost :  6.997959 
Preadiction
 [[181.21983 ]
 [198.19305 ]
 [137.60254 ]
 [102.589714]
 [154.48473 ]
 [182.26485 ]
 [181.21983 ]
 [198.19305 ]
 [137.60254 ]
 [102.589714]]
30 Cost :  6.1137915 
Preadiction
 [[154.41562]
 [182.19437]
 [181.14392]
 [198.11037]
 [137.55191]
 [102.55384]
 [154.41562]
 [182.19437]
 [181.14392]
 [198.11037]]
40 Cost :  7.8364687 
Preadiction
 [[137.45622]
 [102.48495]
 [154.29724]
 [182.06487]
 [181.00972]
 [197.96431]
 [137.45622]
 [102.48495]
 [154.29724]
 [182.06487]]
50 Cost :  6.949282 
Preadiction
 [[181.21446]
 [198.18732]
 [137.62305]
 [102.60949]
 [154.46501]
 [182.2794 ]
 [181.21446]
 [198.18732]
 [137.62305]
 [10

## shiffle_batch
min_after_dequeue = 10000<br>
capacity = min_after_dequeue + 3 * batch_size<br>
example_batch, label_batch = tf.train.shuffle_batch(<br>
    [example, label], batch_size=batch_size, capacity=capacity,<br>
    min_after_dequeue=min_after_dequeue)
